# Manipulação de dados com Pandas II

## Agregando Dados

Vamos utilizar o mesmo dataset da aula anterior de introdução ao Pandas, analisando também os preços dos produtos.

In [ ]:
import pandas as pd

df = pd.read_csv('orders.csv')
df

São funções de agregação básicas: contagem, valores máximo e mínimo, média, desvio padrão e variância dos valores.

In [ ]:
print("count: ", df['price'].count())
print("max: ", df['price'].max())
print("min: ", df['price'].min())
print("média: ", df['price'].mean())
print("std: ", df['price'].std())
print("var: ", df['price'].var())


São estatísticas básicas a mediana (2º quartil) e quartis:

In [ ]:
print("mediana: ", df['price'].median())
print("quantile: ", df['price'].quantile(0))
print("quantile: ", df['price'].quantile(0.25))
print("quantile: ", df['price'].quantile(0.5))
print("quantile: ", df['price'].quantile(0.75))
print("quantile: ", df['price'].quantile(1))

In [ ]:
describe = df.describe().round(2)
describe

Para as colunas que não possuem valores numéricos, é possível estabelecer algumas verificações, como quantos e quais valores únicos cada uma delas possuem:

In [ ]:
print(df.shoe_material.unique())
print(df.shoe_material.nunique())

Para agrupar informações importantes, como, por exemplo, o preço mais alto por tipo de calçado, podemos utilizar a função `groupby`:

In [ ]:
pricey_shoes = df.groupby('shoe_type').price.max()
pricey_shoes

Para transformar o resultado do agrupamento em um novo dataframe, basta acrescentar `reset_index()` no fim:

In [ ]:
pricey_shoes = df.groupby('shoe_type').price.max().reset_index()
pricey_shoes

E assim podemos renomear as colunas, se for necessário:

In [ ]:
pricey_shoes = pricey_shoes.rename(columns={'price': 'max_price'})
pricey_shoes

Para realizar o agrupamento com mais de uma coluna, basta passar uma lista como parâmetro da função `groupby`:

In [ ]:
pricey_shoes_2 = df.groupby(['shoe_type', 'shoe_color']).price.max().reset_index().rename(columns={'price': 'max_price'})

pricey_shoes_2

In [ ]:
pricey_shoes_3 = df.groupby(['shoe_type', 'shoe_color']).price.max().reset_index().rename(columns={'price': 'max_price'})

pricey_shoes_3

In [ ]:
shoe_counts = df.groupby(['shoe_type', 'shoe_color']).id.count().reset_index()

shoe_counts = shoe_counts.rename(columns={'id':'count'})

shoe_counts

Para melhor compreensão, analise outro exemplo: A Biblioteca Municipal tem várias filiais espalhadas pela cidade. Em cada uma delas são  coletados todos os dados de retirada de livros em um DataFrame chamado `checkouts`. 

O DataFrame contém as colunas `location`, `date` e `book_title`. Se quisermos comparar o número total de livros retirados em cada filial, qual código poderíamos usar?

```
checkouts.groupby(['location']).book_title.count().reset_index()
``` 

No exemplo anterior, vimos que muitos dos dados se repetiram. Isso acontece porque fizemos o agrupamento por mais de uma coluna, o que pode ser resolvido estabelecendo um tabela pivô:

```
df.pivot(columns='ColumnToPivot',
         index='ColumnToBeRows',
         values='ColumnToBeValues')
```

In [ ]:
shoe_max_pivot = pricey_shoes_3.pivot(columns='shoe_color', index='shoe_type', values='max_price').reset_index()

shoe_max_pivot

In [ ]:
shoe_counts_pivot = shoe_counts.pivot(columns='shoe_color', index='shoe_type', values='count').reset_index()

shoe_counts_pivot

Ou aplicar duas ou mais métricas usando a função `agg`:

In [ ]:
max_min_shoes = df.groupby('shoe_type').price.agg(['max', 'min'])
max_min_shoes

É possível mesclar o cálculo de estatísticas com o agrupamento dos dados. Por exemplo, para calcular o percentil de 25% (1º quartil) do preço com base no agrupamento por cor do calçado, teremos os sapatos mais baratos assim:

In [ ]:
import numpy as np

cheap_shoes = df.groupby('shoe_color').price.apply(lambda x: np.percentile(x, 25)).reset_index()

cheap_shoes

## Ordenando Dados

In [ ]:
df.sort_values(['price', 'shoe_type'])

## Concatenando Dados

In [ ]:
tabela_1 = pd.read_csv('tabela1.csv')
tabela_1

In [ ]:
tabela_2 = pd.read_csv('tabela2.csv')
tabela_2

In [ ]:
tabela_3 = pd.read_csv('tabela3.csv')
tabela_3

Veremos como este método se comporta em dois cenários:

### Cenário 1

Vejamos como a método se comporta no caso de tabelas com as mesmas colunas. O método gera uma tabela com todas as combinações de colunas.

In [ ]:
pd.concat([tabela_1, tabela_3])

### Cenário 2

Importaremos uma nova tabela chamada tabela_4, idêntica à tabela_3, porém com uma coluna a mais, a coluna ‘Animais’ :

In [ ]:
tabela_4 = pd.read_csv('tabela4.csv')
tabela_4

In [ ]:
t = pd.concat([tabela_1, tabela_4]).reset_index(drop=True)
t

Como podemos ver, as linhas que vieram da tabela_1 receberam ‘NaN’ na coluna ‘Animais’. Previsivelmente, afinal essa coluna só existe na tabela_4.

Aqui, cada linha isolada não faz sentido. Mas o usuário pode filtrar a tabela para obter a informação que deseja.

O código abaixo, por exemplo, nos retorna todos ‘Animais’ relacionados a ‘Nome’ igual a ‘João’, excluindo aqueles valores que aparecem como NaN:

In [ ]:
t[(t['nome'] == 'João') & (t['animais'].notna())]['animais']

Para ajustar os índices, basta incluir o parâmetro `ignore_index`:

In [ ]:
t = pd.concat([tabela_1, tabela_4], ignore_index=True)
t

Para concatenar horizontalmente, basta incluir o parâmetro `axis=1`:

In [ ]:
t = pd.concat([tabela_1, tabela_4], axis=1)
t

## Mesclando Dados

Caso queiramos a interseção exata entre as tabelas: `how = 'inner'`. Obtivemos a interseção entre as duas tabelas. Somente os valores na coluna “Nome” que existem em ambas tabelas aparecem no nosso resultado.

In [ ]:
inner = pd.merge(tabela_1, tabela_2, how='inner', on='nome')
inner

Caso queiramos todas as informações, de ambas tabelas, fazemos um merge `how = 'outer'`. Perceba que "Pedro" não possui dados para “Irmãos”. E “Marcelo” e “Thiago” não possuem dados para “Telefone” e “Carros”. Volte nas tabelas 1 e 2 veja que esse é exatamente o resultado esperado se quisermos todas as informações.

In [ ]:
outer = pd.merge(tabela_1, tabela_2, how='outer', on='nome')
outer

Um merge “left” ou “right” depende de qual tabela você deixa na direita ou esquerda. Para o seguinte cenário faremos um merge do tipo “left”. Mas o mesmo resultado pode ser obtido com um merge “right” trocando a posição das tabelas no método “merge”.

In [ ]:
left = pd.merge(tabela_1, tabela_2, how='left', on='nome')
left

In [ ]:
right = pd.merge(tabela_2, tabela_1, how='right', on='nome')
right

### Cenários para merge left

#### Cenário 1

Suponhamos que para a sua tarefa você deva manter os dados da tabela_1 e ir acrescentando colunas conforme mais dados sobre ‘João’, ‘Pedro’ e ‘Caio’ apareçam. Em outras palavras, somente ‘Nomes’ que existam na tabela_1 serão trazidos da tabela_2. Neste exemplo deixaremos a tabela_1 à esquerda e faremos um merge ‘left’:

In [ ]:
m = pd.merge(tabela_1, tabela_2, how = 'left', on = 'nome')
m

#### Cenário 2

Neste cenário uniremos a tabela_1 à tabela_3. Perceba que a nova tabela, tabela_3, é igual a tabela_1 mas com valores novos para a coluna ‘Carros’.

O que você acha que vai acontecer se tentarmos unir tabela_1 e tabela_3? Com qual valor para ‘Carros’ devemos ficar ao efetuarmos o seguinte merge?

In [ ]:
m = pd.merge(tabela_1, tabela_3, how = 'left', on = 'nome')
m

A nossa chave para o merge sendo ‘nome’, todas outras colunas iguais entre as tabelas são separadas em _x e _y, onde:

_x Corresponde aos valores que existiam na tabela da esquerda (tabela_1).
_y Corresponde aos valores que existiam na tabela da direita (tabela_3).

Esses sufixos podem ser alterados.

In [ ]:
m = pd.merge(tabela_1, tabela_3, how = 'left', on = 'nome').rename(columns = {'carros_x': 'carro antigo', 'carros_y': 'carro novo', 'telefone_x': 'telefone primário', 'telefone_y': 'telefone secundário'})
m